In [1]:
# Kerakli kutubxonalar
import zipfile
import pandas as pd
import os
import seaborn as sns
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import log_loss

In [2]:
zip_path  = '/content/multiclassificationtask.zip'
extract_folder = '/content/multiclassificationtask'


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
 zip_ref.extractall(extract_folder)

os.listdir(extract_folder)


['sample_submission.csv', 'test.csv', 'train.csv']

In [3]:
train_df = pd.read_csv('/content/multiclassificationtask/train.csv')
train_df

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,2178.0,D-penicillamine,16374.0,F,N,N,N,N,0.5,263.0,3.20,43.0,1110.0,106.95,67.0,430.0,9.6,3.0,C
1,1,2644.0,D-penicillamine,17774.0,F,N,N,N,N,0.8,280.0,3.60,22.0,678.0,62.00,80.0,427.0,13.0,3.0,C
2,2,3069.0,Placebo,17844.0,F,N,N,N,N,1.1,408.0,4.40,54.0,2108.0,142.60,137.0,203.0,10.6,3.0,C
3,3,2216.0,Placebo,19221.0,F,N,Y,Y,N,0.8,252.0,3.70,36.0,843.0,55.80,56.0,336.0,9.6,4.0,C
4,4,2256.0,Placebo,21600.0,F,N,N,N,N,4.7,348.0,3.06,464.0,961.0,120.90,146.0,298.0,11.0,2.0,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,1783.0,Placebo,23107.0,F,N,Y,N,S,1.3,243.0,3.76,58.0,944.0,97.00,93.0,233.0,9.9,4.0,C
14996,14996,3492.0,NaN,13149.0,F,NaN,NaN,NaN,N,0.7,NaN,4.03,NaN,NaN,NaN,NaN,273.0,10.3,2.0,C
14997,14997,1170.0,NaN,22646.0,F,NaN,NaN,NaN,N,0.8,NaN,3.14,NaN,NaN,NaN,NaN,280.0,10.6,3.0,C
14998,14998,1433.0,Placebo,21828.0,F,N,N,N,N,0.6,280.0,4.04,52.0,843.0,28.38,189.0,334.0,10.0,2.0,C


In [4]:
test_df = pd.read_csv('/content/multiclassificationtask/test.csv')
test_df

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,15000,22646.0,NaN,22646.0,F,NaN,NaN,NaN,N,1.4,NaN,3.34,NaN,NaN,NaN,NaN,190.0,11.0,4.0
1,15001,2149.0,NaN,18993.0,F,NaN,NaN,NaN,N,0.9,NaN,3.24,NaN,NaN,NaN,NaN,233.0,9.0,2.0
2,15002,3850.0,D-penicillamine,16050.0,M,N,N,N,N,1.0,460.0,3.64,75.0,2148.0,102.30,118.0,172.0,10.6,3.0
3,15003,2419.0,D-penicillamine,20354.0,F,N,N,N,N,0.6,258.0,3.96,31.0,642.0,70.00,83.0,336.0,10.8,2.0
4,15004,1086.0,NaN,23376.0,F,NaN,NaN,NaN,N,4.4,NaN,4.40,NaN,NaN,NaN,NaN,447.0,10.3,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,2106.0,Placebo,16728.0,F,N,N,N,N,1.1,1128.0,3.35,91.0,815.0,127.10,166.0,421.0,9.6,1.0
9996,24996,186.0,D-penicillamine,13073.0,M,N,Y,N,N,0.6,267.0,3.02,225.0,728.0,120.90,90.0,142.0,12.3,4.0
9997,24997,3458.0,D-penicillamine,20296.0,F,N,N,N,N,0.9,NaN,3.35,136.0,1995.0,128.00,NaN,243.0,10.6,2.0
9998,24998,2891.0,D-penicillamine,20254.0,F,N,N,N,N,0.8,NaN,3.75,63.0,674.0,26.35,NaN,325.0,10.0,1.0


In [5]:
train_df.shape

(15000, 20)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           8494 non-null   object 
 3   Age            15000 non-null  float64
 4   Sex            15000 non-null  object 
 5   Ascites        8502 non-null   object 
 6   Hepatomegaly   8492 non-null   object 
 7   Spiders        8491 non-null   object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    6701 non-null   float64
 11  Albumin        15000 non-null  float64
 12  Copper         8399 non-null   float64
 13  Alk_Phos       8488 non-null   float64
 14  SGOT           8486 non-null   float64
 15  Tryglicerides  6666 non-null   float64
 16  Platelets      14436 non-null  float64
 17  Prothrombin    14984 non-null  float64
 18  Stage 

In [7]:
train_df.describe()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,15000.000000,15000.000000,15000.000000,15000.000000,6701.000000,15000.000000,8399.000000,8488.000000,8486.000000,6666.000000,14436.000000,14984.000000,15000.000000
mean,7499.500000,1971.721267,19298.658267,1.845433,329.177302,3.526103,75.647339,1618.598233,106.483631,111.387886,252.917983,10.626111,3.024800
std,4330.271354,1333.733576,3797.580260,2.674859,180.320802,0.372268,74.830472,1767.240078,57.206017,55.056133,94.049175,0.731416,0.871303
min,0.000000,3.000000,400.000000,0.200000,120.000000,0.500000,2.000000,3.400000,0.900000,19.000000,32.000000,9.000000,1.000000
25%,3749.750000,1095.000000,16658.000000,0.600000,242.000000,3.290000,31.000000,720.000000,71.000000,80.000000,181.000000,10.100000,2.000000
50%,7499.500000,1786.000000,19544.000000,0.900000,280.000000,3.580000,52.000000,1072.000000,97.650000,99.000000,248.000000,10.600000,3.000000
75%,11249.250000,2635.000000,22347.000000,1.800000,364.000000,3.770000,89.000000,1664.000000,130.200000,133.000000,311.000000,11.000000,4.000000
max,14999.000000,38320.000000,129398.000000,28.000000,2078.000000,4.640000,662.000000,13862.400000,2653.000000,1669.000000,1024.000000,18.000000,4.000000


In [8]:
train_df.isnull().sum()

,0
id,0
N_Days,0
Drug,6506
Age,0
Sex,0
Ascites,6498
Hepatomegaly,6508
Spiders,6509
Edema,0
Bilirubin,0


In [9]:
train_df.Drug.value_counts()

,count
Drug,
D-penicillamine,4610
Placebo,3884


In [10]:
train_df.Ascites.value_counts()

,count
Ascites,
N,8116
Y,386


In [11]:
train_df.drop_duplicates(inplace=True)

In [12]:
train_df['age_in_years']=(train_df['Age']/365).astype(int)
train_df.drop('Age', axis=1, inplace=True)
train_df.head()

,id,N_Days,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status,age_in_years
0,0,2178.0,D-penicillamine,F,N,N,N,N,0.5,263.0,3.20,43.0,1110.0,106.95,67.0,430.0,9.6,3.0,C,44
1,1,2644.0,D-penicillamine,F,N,N,N,N,0.8,280.0,3.60,22.0,678.0,62.00,80.0,427.0,13.0,3.0,C,48
2,2,3069.0,Placebo,F,N,N,N,N,1.1,408.0,4.40,54.0,2108.0,142.60,137.0,203.0,10.6,3.0,C,48
3,3,2216.0,Placebo,F,N,Y,Y,N,0.8,252.0,3.70,36.0,843.0,55.80,56.0,336.0,9.6,4.0,C,52
4,4,2256.0,Placebo,F,N,N,N,N,4.7,348.0,3.06,464.0,961.0,120.90,146.0,298.0,11.0,2.0,D,59


In [13]:
train_df=train_df[train_df['Status'] != 'Y']
X=train_df.drop(columns=['id','Status'])
y=train_df['Status']

In [14]:
test_df['age_in_years']=(test_df['Age']/365).astype(int)
test_df.drop(['Age','id'], axis=1, inplace=True)
test_df.head()

,N_Days,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,age_in_years
0,22646.0,NaN,F,NaN,NaN,NaN,N,1.4,NaN,3.34,NaN,NaN,NaN,NaN,190.0,11.0,4.0,62
1,2149.0,NaN,F,NaN,NaN,NaN,N,0.9,NaN,3.24,NaN,NaN,NaN,NaN,233.0,9.0,2.0,52
2,3850.0,D-penicillamine,M,N,N,N,N,1.0,460.0,3.64,75.0,2148.0,102.3,118.0,172.0,10.6,3.0,43
3,2419.0,D-penicillamine,F,N,N,N,N,0.6,258.0,3.96,31.0,642.0,70.0,83.0,336.0,10.8,2.0,55
4,1086.0,NaN,F,NaN,NaN,NaN,N,4.4,NaN,4.40,NaN,NaN,NaN,NaN,447.0,10.3,2.0,64


In [15]:
test_df.isnull().sum()

,0
N_Days,0
Drug,4398
Sex,0
Ascites,4391
Hepatomegaly,4402
Spiders,4404
Edema,0
Bilirubin,0
Cholesterol,5647
Albumin,0


Pipeline dan o'tqizish

In [16]:
frequent_cols=['Ascites','Spiders','Sex','Edema']
ffill_cols = ['Drug', 'Hepatomegaly']
num_cols = X.select_dtypes(include=['number']).columns.tolist()

In [17]:
class ForwardFillImputer(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return pd.DataFrame(X).ffill().values

In [18]:
most_freq_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])
ffill_pipeline = Pipeline([
    ('ffill', ForwardFillImputer()),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [20]:
full_pipeline = ColumnTransformer([
    ('most_freq_cat', most_freq_pipeline, frequent_cols),
    ('ffill_cat', ffill_pipeline, ffill_cols),
    ('num', num_pipeline, num_cols)
])

In [21]:
X_transformed=full_pipeline.fit_transform(X)
X_transformed[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.15478384, -0.50302697, -0.31691144, -0.87625475,
       -0.38883121, -0.19997958,  0.09943872, -1.00801519,  1.92120314,
       -1.40389741, -0.02838968, -0.81986736])

Trainset va Test set

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

ML
LogisticRegression Modeli

In [23]:
LR_model=LogisticRegression(multi_class='multinomial', solver='lbfgs')
LR_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(multi_class='multinomial')

In [24]:
# Modelni baholash
y_pred_proba = LR_model.predict_proba(X_test)
loss = log_loss(y_test, y_pred_proba)
print("Log Loss:", loss)

Log Loss: 0.4657376850047918


Random Forest Modeli

In [25]:
RF_model=RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
RF_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=42)

In [26]:
y_pred_proba = RF_model.predict_proba(X_test)
loss = log_loss(y_test, y_pred_proba)
print("Log Loss: " ,loss)

Log Loss:  0.4317231627947693


XGBoost Modeli

In [27]:
le=LabelEncoder()
y_train_encoded=le.fit_transform(y_train)
y_test_encoded=le.fit_transform(y_test)

In [28]:
XGB_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
XGB_model.fit(X_train, y_train_encoded)
y_proba=XGB_model.predict_proba(X_test)


loss = log_loss(y_test_encoded, y_proba)
print("Log Loss: " ,loss)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:39:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Log Loss:  0.39455271972198847


Submission csv

In [30]:
test_transformed=full_pipeline.transform(test_df)

In [31]:
test_predictions = XGB_model.predict_proba(test_transformed)

if 'id' in test_df.columns:
    submission = pd.DataFrame({'id': test_df['id']})
else:
    submission = pd.DataFrame({'id': range(15000, 15000+len(test_df))})

submission[['Status_C', 'Status_CL', 'Status_D']] = test_predictions

submission.to_csv('submission.csv', index=False)
print(submission)

         id  Status_C  Status_CL  Status_D
0     15000  0.431272   0.000851  0.567877
1     15001  0.968492   0.005400  0.026108
2     15002  0.595574   0.001505  0.402921
3     15003  0.989028   0.000078  0.010893
4     15004  0.199273   0.227788  0.572940
...     ...       ...        ...       ...
9995  24995  0.982037   0.004179  0.013784
9996  24996  0.005708   0.000538  0.993754
9997  24997  0.936188   0.001744  0.062068
9998  24998  0.984116   0.000127  0.015757
9999  24999  0.734442   0.024510  0.241048

[10000 rows x 4 columns]
